In [31]:
import numpy as np
from numpy import array
import pandas as pd
from pandas import DataFrame

import datetime

## 补齐时频数据

##### 见rainstation_seperate.ipynb

## 时频数据->日频

In [32]:
rain_st = pd.read_csv('/home/lzy/lab_env/Project/太湖流域数据/处理数据/时频/雨量-湖西区3-整编后.csv',index_col=0)
print(rain_st.shape)
rain_st.columns

FileNotFoundError: [Errno 2] No such file or directory: '/home/lzy/lab_env/Project/太湖流域数据/处理数据/时频/雨量-浙西区3-整编后.csv'

In [ ]:
rain_st.iloc[0,1]

'2012-01-08 02:00:00'

In [ ]:
for n in range(0, len(rain_st['TM'])):
    # timestrip = rain_st['TM'].iloc[n].replace('/','-')
    timestrip = rain_st.iloc[n,1].replace('/','-')
    timestrip = datetime.datetime.strptime(timestrip, "%Y-%m-%d %H:%M:%S")
    if timestrip.hour == 8:
        # print(j)
        break
n

6

In [ ]:
for i in range(n, len(rain_st['TM']), 24):
    if rain_st.iloc[i,3] == 0:
        sum = rain_st.iloc[i-23:i+1,2].sum()
        # print(sum)
        rain_st.iloc[i,3] = sum
rain_st.iloc[:,0:4].shape

(95353, 4)

In [ ]:
type(rain_st.iloc[14,1])
len(rain_st['TM'])

95353

In [ ]:
# pd.set_option('display.max_rows',None)
rain_st.iloc[:,[5,7]].dropna()

,TM.1,DYP.1
0,2012-01-08 03:00:00,0.0
1,2012-01-08 04:00:00,0.0
2,2012-01-08 05:00:00,0.0
3,2012-01-08 06:00:00,0.0
4,2012-01-08 07:00:00,0.0
...,...,...
89404,2022-03-21 07:00:00,0.0
89405,2022-03-21 08:00:00,37.0
89406,2022-03-21 09:00:00,0.0
89407,2022-03-21 10:00:00,0.0


In [ ]:
time_list = []
for i in range(n, len(rain_st['TM']), 24):
    time_list.append(rain_st.iloc[i,1])
time_df = pd.DataFrame(time_list,columns=['TM']).dropna()
print(time_df)
# rain_st.iloc[:,[1,3]].dropna()
complete = rain_st.iloc[:,[1,3]].merge(time_df,how='right',sort=True)
complete['TM'] = pd.to_datetime(complete['TM'])
complete = complete.sort_values(by='TM')
complete
# complete.to_csv('/home/lzy/lab_env/Project/rainfall_station/day_value.csv')

                       TM
0     2012-01-08 08:00:00
1     2012-01-09 08:00:00
2     2012-01-10 08:00:00
3     2012-01-11 08:00:00
4     2012-01-12 08:00:00
...                   ...
3962  2022-11-13 08:00:00
3963  2022-11-14 08:00:00
3964  2022-11-15 08:00:00
3965  2022-11-16 08:00:00
3966  2022-11-17 08:00:00

[3967 rows x 1 columns]


,TM,DYP
0,2012-01-08 08:00:00,0.0
1,2012-01-09 08:00:00,0.5
2,2012-01-10 08:00:00,0.0
3,2012-01-11 08:00:00,0.0
4,2012-01-12 08:00:00,0.0
...,...,...
3962,2022-11-13 08:00:00,0.0
3963,2022-11-14 08:00:00,0.0
3964,2022-11-15 08:00:00,0.5
3965,2022-11-16 08:00:00,0.0


In [ ]:
# rain_st_day = pd.DataFrame(columns=['STCD','TM','DYP'])
# rain_st_day=rain_st.iloc[17,4]
# rain_st_day
len(rain_st['TM'])

95353

In [ ]:
final_list = []
for i in range(1, rain_st.shape[1], 4): #rain_st.shape[1] = 25
    for n in range(0, len(rain_st['TM'])):
        timestrip = rain_st.iloc[n,i].replace('-','/')
        timestrip = datetime.datetime.strptime(timestrip, "%Y/%m/%d %H:%M:%S")
        if timestrip.hour == 8:
            # print(j)
            break
    # print(n)
    for j in range(n, len(rain_st.iloc[:,i]), 24):
        if rain_st.iloc[j,i+2] == 0:
            dp = j-23
            if dp < 0:
                dp = 0
            sum = rain_st.iloc[dp:j+1,i+1].sum()
            # print(sum)
            rain_st.iloc[j,i+2] = sum
            # print(sum)
        # print(rain_st.iloc[j,i-1:i+3])
    time_list = []

    for k in range(n, len(rain_st['TM']), 24):
        time_list.append(rain_st.iloc[k,i])
    time_df = pd.DataFrame(time_list,columns=['TM']).dropna()
    new_col = rain_st.iloc[:,[i-1,i,i+2]].rename(columns={rain_st.columns[i-1]:'STCD',rain_st.columns[i]:'TM',rain_st.columns[i+2]:'DYP'})
    complete = new_col.merge(time_df,how='right',sort=True)
    complete['TM'] = pd.to_datetime(complete['TM'])
    complete = complete.sort_values(by='TM')
    complete.reset_index(drop=True,inplace=True)
    final_list.append(complete)
    # rain_st_day = pd.concat([rain_st_day, DYP], axis=1)


In [ ]:
for i in range(len(final_list)):
    print(final_list[i])

            STCD                  TM   DYP
0     63000200.0 2012-01-08 08:00:00   3.0
1     63000200.0 2012-01-09 08:00:00   0.5
2     63000200.0 2012-01-10 08:00:00   0.0
3     63000200.0 2012-01-11 08:00:00   0.0
4     63000200.0 2012-01-12 08:00:00   0.0
...          ...                 ...   ...
3962  63000200.0 2022-11-13 08:00:00   0.0
3963  63000200.0 2022-11-14 08:00:00   0.0
3964  63000200.0 2022-11-15 08:00:00   0.5
3965  63000200.0 2022-11-16 08:00:00   0.0
3966  63000200.0 2022-11-17 08:00:00  17.1

[3967 rows x 3 columns]
            STCD                  TM   DYP
0     63000300.0 2012-01-08 08:00:00   3.0
1     63000300.0 2012-01-09 08:00:00   1.0
2     63000300.0 2012-01-10 08:00:00   0.0
3     63000300.0 2012-01-11 08:00:00   0.0
4     63000300.0 2012-01-12 08:00:00   0.0
...          ...                 ...   ...
3721  63000300.0 2022-03-17 08:00:00   9.5
3722  63000300.0 2022-03-18 08:00:00  21.0
3723  63000300.0 2022-03-19 08:00:00   0.0
3724  63000300.0 2022-03-20 0

In [ ]:
rain_st_day = pd.concat(final_list,axis=1)
rain_st_day

,STCD,TM,DYP,STCD,TM,DYP,STCD,TM,DYP,STCD,...,DYP,STCD,TM,DYP,STCD,TM,DYP,STCD,TM,DYP
0,63000200.0,2012-01-08 08:00:00,3.0,63000300.0,2012-01-08 08:00:00,3.0,63000600.0,2012-01-08 08:00:00,3.0,63001000.0,...,2.0,63008800.0,2012-01-08 08:00:00,1.5,63009900.0,2012-01-08 08:00:00,1.5,63010800.0,2021-07-28 08:00:00,35.5
1,63000200.0,2012-01-09 08:00:00,0.5,63000300.0,2012-01-09 08:00:00,1.0,63000600.0,2012-01-09 08:00:00,1.5,63001000.0,...,1.0,63008800.0,2012-01-09 08:00:00,1.5,63009900.0,2012-01-09 08:00:00,2.5,63010800.0,2021-07-29 08:00:00,0.0
2,63000200.0,2012-01-10 08:00:00,0.0,63000300.0,2012-01-10 08:00:00,0.0,63000600.0,2012-01-10 08:00:00,0.0,63001000.0,...,0.0,63008800.0,2012-01-10 08:00:00,0.0,63009900.0,2012-01-10 08:00:00,0.0,63010800.0,2021-07-30 08:00:00,0.0
3,63000200.0,2012-01-11 08:00:00,0.0,63000300.0,2012-01-11 08:00:00,0.0,63000600.0,2012-01-11 08:00:00,0.0,63001000.0,...,0.0,63008800.0,2012-01-11 08:00:00,0.0,63009900.0,2012-01-11 08:00:00,0.0,63010800.0,2021-07-31 08:00:00,3.0
4,63000200.0,2012-01-12 08:00:00,0.0,63000300.0,2012-01-12 08:00:00,0.0,63000600.0,2012-01-12 08:00:00,0.0,63001000.0,...,0.0,63008800.0,2012-01-12 08:00:00,0.0,63009900.0,2012-01-12 08:00:00,0.0,63010800.0,2021-08-01 08:00:00,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3969,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,...,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaT,NaN
3970,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,...,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaT,NaN
3971,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,...,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaT,NaN
3972,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,...,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaT,NaN


In [ ]:
rain_st_day.to_csv('/home/lzy/lab_env/Project/太湖流域数据/处理数据/日频//雨量-浙西区3-日频.csv')